In [2]:
import openai
import pandas as pd

In [ ]:
from livebench.model.model_adapter import (
    get_conversation_template, 
)

In [ ]:
model = 'Qwen/Qwen2.5-7B-Instruct'
question = '### Instructions: You are an expert Python programmer. You will be given a question (problem specification) and will generate a correct Python program that matches the specification and passes all tests. You will NOT return anything except for the program.\n### Question:\nTakahashi, a young baseball enthusiast, has been a very good boy this year, so Santa has decided to give him a bat or a glove, whichever is more expensive.\nIf a bat costs B yen and a glove costs G yen (B\\neq G), which one will Santa give to Takahashi?\n\nInput\n\nThe input is given from Standard Input in the following format:\nB G\n\nOutput\n\nIf Santa gives Takahashi a bat, print Bat; if Santa gives him a glove, print Glove.\n\nConstraints\n\n\n- B and G are different integers between 1 and 1000, inclusive.\n\nSample Input 1\n\n300 100\n\nSample Output 1\n\nBat\n\nThe bat is more expensive than the glove, so Santa will give Takahashi the bat.\n\nSample Input 2\n\n334 343\n\nSample Output 2\n\nGlove\n\nThe glove is more expensive than the bat, so Santa will give Takahashi the glove.\n\n### Format: Read the inputs from stdin solve the problem and write the answer to stdout (do not directly test on the sample inputs). Enclose your code within delimiters as follows.\n```python\n# YOUR CODE HERE\n```\n\n### Answer: (use the provided format with backticks)\n\n'

conv = get_conversation_template(model)
conv.append_message(conv.roles[0], question)
conv.append_message(conv.roles[1], None)
temperature = 0.7
max_tokens = 10


In [ ]:
conv.to_openai_api_messages()

In [ ]:
messages = conv.to_openai_api_messages()
messages

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
t2 = tokenizer.apply_chat_template(messages[1:], tokenize=False)
t2

In [ ]:
t1 = tokenizer.apply_chat_template(messages[:1], tokenize=False)
t1

In [ ]:
openai.api_base = "http://localhost:32000/v1"
openai.api_key = "mykey"
messages = conv.to_openai_api_messages()
response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    n=1,
    temperature=temperature,
    max_tokens=max_tokens,
    logprobs=True,
    top_logprobs=5,
)


In [ ]:
response

In [ ]:
model

In [31]:
messages = tokenizer.apply_chat_template(
    conv, tokenize=False
)

In [39]:
import requests

response = requests.post(
    "http://localhost:32000/generate",
    json=dict(
        text=messages,
        rid="0",
        return_logprob=True,
        top_logprobs_num=2, 
        return_text_in_logprobs=True,
        sampling_params=dict(
            skip_special_tokens=True,
            spaces_between_special_tokens=True,
            temperature=0.7,
            max_new_tokens=10,
        ),
    )
)

In [ ]:
response

In [ ]:
messages

In [ ]:
response.json()

In [ ]:
response

In [ ]:

openai.api_base = "http://localhost:32000/v1"
openai.api_key = "mykey"
messages = conv.to_openai_api_messages()
response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    n=1,
    temperature=temperature,
    max_tokens=max_tokens,
    logprobs=True,
    top_logprobs=5,
    stream=True,
    stream_options=dict(include_usage=True,)
)
responses = list(response)


In [ ]:
responses[0]

In [ ]:
responses[-1]["usage"].to_dict()

In [ ]:
def extract_top_logprobs(x):
    results = []
    for item in x:
        results.append(
            (item["token"], item["logprob"])
        )
    return results


In [228]:
def extract_logprobs_info(responses):
    xxx = [
        (
            item["token"], item["logprob"], extract_top_logprobs(item["top_logprobs"])
        )
        for i in range(len(responses))
        for item in responses[i]["choices"][0]["logprobs"]["content"]
    ]
    return xxx

def get_logprobs_info(responses):
    return extract_logprobs_info(responses[1:])


In [ ]:
get_logprobs_info(responses)

In [160]:
w = response.to_dict()

In [161]:
def extract_logprobs(w):
    return [
        w['choices'][0]['logprobs']['content'][i]["logprob"]
        for i in range(len(w['choices'][0]['logprobs']['content']))
    ]

logprobs = extract_logprobs(w)


In [ ]:
# w['choices'][0]['logprobs']['content'][13]["token"]
w['choices'][0]['logprobs']['content']#[20]["top_logprobs"]

In [ ]:
w

In [ ]:
logprobs

In [83]:
from livebench.common import get_categories_tasks, load_questions
categories, tasks = get_categories_tasks("live_bench")

In [84]:
category_name = "coding"
task_name = "coding_completion"
release_set = {'2024-07-26', '2024-06-24', '2024-08-31'}
# questions = load_questions(categories[category_name], release_set, task_name, None, None)

In [85]:
with open("./df_raw.csv", "r") as file:
    df = pd.read_csv(file)
df = df[df['task'] == task_name] 
df = df[df['category'] == category_name]
df['score'] = df['score'] / 100

In [ ]:
df.columns

In [ ]:
df_grouped = df.groupby([
    "model", "task", "category", "question_id", "question_tok_len",

]).agg(
    record_count=pd.NamedAgg(column="score", aggfunc="count"),
    score_sum=pd.NamedAgg(column="score", aggfunc="sum"),
    score_list=pd.NamedAgg(column="score", aggfunc=list),
    answer_tok_len_list=pd.NamedAgg(column="llm_answer_tok_len", aggfunc=list),
).reset_index()
df_grouped['score_sum'] /= 100
df_grouped['score_mean'] = df_grouped['score_sum'] / df_grouped['record_count'] * 100
df_grouped


In [ ]:
# print("\n########## All Tasks ##########")
df_1 = df[["model", "score", "task"]]
df_1 = df_1.groupby(["model", "task"]).mean()
df_1 = pd.pivot_table(df_1, index=['model'], values = "score", columns=["task"], aggfunc="sum")
df_1 = df_1.round(3)
# print(df_1.sort_values(by="model")[:60])
# df_1.to_csv('all_tasks.csv')

In [ ]:
df_1

In [11]:
# import sglang as sgl
# from sglang import function, set_default_backend, RuntimeEndpoint
# from sglang.lang.interpreter import ProgramState

# llm = RuntimeEndpoint("http://localhost:34000")
# set_default_backend(llm)

from transformers import AutoTokenizer
model = 'Qwen/Qwen2.5-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(
    model,
    use_fast=True,
    trust_remote_code=True,
)

from fastchat.conversation import Conversation, get_conv_template
get_conv_template("qwen-7b-chat")
question = '### Instructions: You are an expert Python programmer. You will be given a question (problem specification) and will generate a correct Python program that matches the specification and passes all tests. You will NOT return anything except for the program.\n### Question:\nTakahashi, a young baseball enthusiast, has been a very good boy this year, so Santa has decided to give him a bat or a glove, whichever is more expensive.\nIf a bat costs B yen and a glove costs G yen (B\\neq G), which one will Santa give to Takahashi?\n\nInput\n\nThe input is given from Standard Input in the following format:\nB G\n\nOutput\n\nIf Santa gives Takahashi a bat, print Bat; if Santa gives him a glove, print Glove.\n\nConstraints\n\n\n- B and G are different integers between 1 and 1000, inclusive.\n\nSample Input 1\n\n300 100\n\nSample Output 1\n\nBat\n\nThe bat is more expensive than the glove, so Santa will give Takahashi the bat.\n\nSample Input 2\n\n334 343\n\nSample Output 2\n\nGlove\n\nThe glove is more expensive than the bat, so Santa will give Takahashi the glove.\n\n### Format: Read the inputs from stdin solve the problem and write the answer to stdout (do not directly test on the sample inputs). Enclose your code within delimiters as follows.\n```python\n# YOUR CODE HERE\n```\n\n### Answer: (use the provided format with backticks)\n\n'

conv = get_conv_template("qwen-7b-chat") # get_conversation_template(model)
conv.append_message(conv.roles[0], question)
conv.append_message(conv.roles[1], None)
temperature = 0.7
max_tokens = 1000

conv.to_openai_api_messages()
import openai

_response = None

def callback_func(response):
    global _response
    _response = response

def chat_completion_openai(model, conv, temperature, max_tokens, api_dict=None, 
                           use_logprobs=True, return_response=False):
    if api_dict is not None:
        openai.api_base = api_dict["api_base"]
        openai.api_key = api_dict["api_key"]
    output = None
    
    messages = conv.to_openai_api_messages()
    import sys
    import traceback

    def tracefunc(frame, event, arg):
        if event == "call":
            print(f"Calling function: {frame.f_code.co_name} in file {frame.f_code.co_filename}")
            # print("Stack trace:")
            # traceback.print_stack(frame)
        return tracefunc

    # sys.settrace(tracefunc)
    sys.settrace(None)
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        n=1,
        temperature=temperature,
        max_tokens=max_tokens,
        logprobs=True,
        top_logprobs=2,
        return_raw_response_callback=callback_func,
    )
    sys.settrace(None)
    return response
    # output = response["choices"][0]["message"]["content"]
    # if return_response:
    #     return response
    # else:
    #     return output

In [12]:
resp = chat_completion_openai(model, conv, temperature, max_tokens, api_dict=dict(
    api_base="http://localhost:34000/v1",
    api_key="mykey",
))
output = resp.choices[0].message.content
print(output)

create: <function callback_func at 0x7f27617f3e20>
```python
# YOUR CODE HERE
b, g = map(int, input().split())
if b > g:
    print("Bat")
else:
    print("Glove")
```


In [13]:
_response.__dict__

{'_headers': {'date': 'Mon, 25 Nov 2024 19:54:49 GMT', 'server': 'uvicorn', 'content-length': '8610', 'content-type': 'application/json'},
 'data': {'id': 'f88d1d82d132489a9a18501621d4ef22',
  'object': 'chat.completion',
  'created': 1732564490,
  'model': 'Qwen/Qwen2.5-7B-Instruct',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': '```python\n# YOUR CODE HERE\nb, g = map(int, input().split())\nif b > g:\n    print("Bat")\nelse:\n    print("Glove")\n```'},
    'logprobs': {'content': [{'token': '```',
       'bytes': [96, 96, 96],
       'logprob': -1.1920928244535389e-07,
       'top_logprobs': [{'token': '```',
         'bytes': [96, 96, 96],
         'logprob': -1.1920928244535389e-07},
        {'token': ' ```', 'bytes': [32, 96, 96, 96], 'logprob': -16.625}]},
      {'token': 'python',
       'bytes': [112, 121, 116, 104, 111, 110],
       'logprob': -9.059865078597795e-06,
       'top_logprobs': [{'token': 'python',
         'bytes': [112, 121, 116,

In [ ]:
resp